In [84]:
import importlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

tips = sns.load_dataset('tips')
plt.style.use('dark_background')

In [567]:
colors = ["#59ffc5", "#19ffaf", "#19ffaf", "#00eaff", "#ffed4f", "#f67dff", "#ff59db", "#ff59db", "#fffba6"]

In [579]:
import neona
importlib.reload(neona)

<module 'neona' from 'C:\\Users\\GIGABYTE\\Desktop\\3 Month Course\\PRACTICE\\Neona-project\\neona.py'>

# 

# BAR

In [512]:
def plot_configure(spines_yn= True, grid_kws= {}, figsize= None):
    fig, ax = plt.subplots(figsize= figsize)
    for spine in ['left', 'right', 'top', 'bottom']:
        ax.spines[spine].set_visible(spines_yn)
    return fig, ax

In [513]:
def get_point(fig, point_size):
    fig_width_x_height = fig.get_figwidth() * fig.get_figheight()
    
    point_size *= 1_000
    patch_size = (point_size * fig_width_x_height) // 150
    decrease = (500 * patch_size) // 10_000
    point = (50 * patch_size) // 10_000

    size = np.arange(patch_size, 0, -decrease); gap = len(size)
    alpha = np.linspace(.05, .5, gap) ** 3
    return size, alpha, point

In [514]:
def get_lw():
    a= np.arange(15, -1, -4); gap = len(a)
    b= np.arange(0, 1, (1/gap)-.1)
    return a, b

In [576]:
def barplot(x, y, spines= True, figsize= None, point_size= 2, color= "#59ffc5", point_marker= 'o', font_dict= dict(), font_offset_xy= (0, 0), show_values= True):
    fig, ax = plot_configure(spines_yn= spines, figsize= figsize)
    sizes, alphas, point = get_point(fig, point_size)
    a, b = get_lw()
    
    for xi, yi in zip(x, y):
        for width, alpha in zip(a, b):
            ax.plot([xi, xi], [0, yi], lw= width, alpha= alpha, color= color)
        ax.plot([xi, xi], [0, yi], lw= 1, alpha= 1, color= color)
        
        for size, alpha in zip(sizes, alphas):
            ax.scatter(xi, yi, s= size, alpha= alpha, color= color, marker= point_marker)
        ax.scatter(xi, yi, s= point, alpha= 1, color= color, marker= point_marker)
        
    if show_values:
        x = plt.xticks()[0]
        for xi, yi in zip(x, y):
            ox, oy = font_offset_xy
            ax.text(xi + ox, yi + oy, str(yi), **font_dict)
            
    return ax

# 

# LINE